## Examples with ```CopasiProcess``` using ``builder.Builder`` and ``biosimulator_processes.data_model``

In [1]:
import sys 

sys.path.insert(0, '..')

In [2]:
import os 
from process_bigraph import pp, pf 
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder

In [4]:
from bigraph_schema.registry import map_type_to_pydantic

In [10]:
c = {'process_name': 'Rose'}


StringTree = map_type_to_pydantic('tree[string]')

tree = StringTree(a=10, b=20, c=30)

tree

#### Initialize a fresh instance of the `BiosimulatorBuilder`, which is inherently configured by `biosimulator_processes.CORE` in the top level.

In [ ]:
b = BiosimulatorBuilder()

#### Example 1: Add a simple `CopasiProcess`-based composite to the bigraph, specifying parameter values through the `data_model`. For now, let's try loading a biomodel and adding a change to initial concentration of a species. We will then add another copasi process instance, whose model is this time loaded from a file (to demonstrate the data model)

Let's define a model instance from a file.

In [3]:
model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

In [ ]:
# Uncomment if you wish to inspect the species referenced below
# model_from_file = load_model(model_filepath)
# get_species(model=model_from_file)

In [ ]:
# 1. specify model changes (we know the model specs as we are providing the file. TODO: allow users to introspect the model before change)
process_model_changes = TimeCourseModelChanges(
    species_changes=[SpeciesChanges(name='ADP', initial_concentration=2.1**3)]
)


pp(process_model_changes.model_dump())

In [ ]:
# 2. define the model schema to be used by the composite process (one of the copasiprocess config parameters)

process_model_from_file = TimeCourseModel(
    model_id='copasi_process_from_file',
    model_source=model_filepath,
    model_name='Simple Composite Process from File',
    model_changes=process_model_changes
)


pp(process_model_from_file.model_dump())

In [ ]:
# 3. Define config schema to be used as 'config' parameter of Process constructor
process_config_from_file = CopasiProcessConfigSchema(
    method='deterministic',
    model=process_model_from_file,
    process_name='simple_copasi_from_file'
)


pp(process_config_from_file.model_dump())

In [ ]:
process_config_from_file.process_name, process_config_from_file.method, process_config_from_file.model

In [ ]:
# 4. Add the process instance by the name of 'simple_copasi' to the builder


b[process_config_from_file.process_name].add_process(
    name='CopasiProcess',
    model=process_config_from_file.model,
    method=process_config_from_file.method
)

Let's now add a biomodel with adjusted params (we know ahead of time)

In [ ]:
# Uncomment if you wish to introspect the model referred below
# from basico import *
# from basico import biomodels
# 
# 
# biomodel_id = 'BIOMD0000000861'
# biomodel = load_biomodel(biomodel_id)
# get_species(model=biomodel)

In [ ]:
# reaction_names = get_reactions(model=biomodel)
# 
# reaction_names

In [ ]:
rparam = ReactionParameter(
    parameter_name='EpoRpRJAK2',
    value=21312.2
)

rparam2 = ReactionParameter(
    parameter_name='EpoRJAK2',
    value=21312.2
)

rc1 = ReactionChanges(
    reaction_name='reaction_1',
    parameter_changes=[rparam, rparam2]
)

rc2 = ReactionChanges(
    reaction_name='reaction_11',
    reaction_scheme='A + B -> C'
)

In [ ]:
biomodel_process_changes = TimeCourseModelChanges(
    reaction_changes=[rc1, rc2]
)

In [ ]:
pp(biomodel_process_changes.model_dump())

In [ ]:
biomodel_id = 'BIOMD0000000861'


biomodel_process_model = TimeCourseModel(
    model_id=biomodel_id,
    model_source=biomodel_id,
    model_name='composite_from_biomodel',
    model_changes=biomodel_process_changes
)

pp(biomodel_process_model.model_dump())

In [ ]:
biomodel_process_config = CopasiProcessConfigSchema(
    process_name='copasi_process_from_biomodel',
    model=biomodel_process_model,
    method='stochastic'
)


pp(biomodel_process_config.model_dump())

Add the biomodel config to the bigraph

In [ ]:
b[biomodel_process_config.process_name].add_process(
    name='CopasiProcess',
    model=biomodel_process_config.model,
    method=biomodel_process_config.method
)

Visualize the added processes

In [ ]:
b.visualize()

Connect the processes together with the default store name:

In [ ]:
b.connect_all(append_to_store_name='_store')

b

In [ ]:
b.visualize()

Run composite 

In [ ]:
composite = b.generate()

In [ ]:
composite.run(10)

Generate a single process with no model changes from the model_filepath

In [6]:
b2 = BiosimulatorBuilder()

In [7]:
single_process_model = TimeCourseModel(
    model_source=ModelFilepath(value=model_filepath),
    model_id='BIOMD0000000061'
)

In [8]:
single_process_config = CopasiProcessConfigSchema(
    process_name='single_copasi_process',
    model=single_process_model,
    method='stochastic'
)

pp(single_process_config.model_dump())

In [9]:
b2[single_process_config.process_name].add_process(
    name='CopasiProcess',
    model=single_process_config.model,
    method=single_process_config.method
)

In [10]:
b2.connect_all(append_to_store_name='_store')

In [11]:
b2.visualize()

In [12]:
pp(b2)

In [13]:
composite2 = b2.generate()

In [14]:
# TODO: try different solvers for this: certainly stochastic

composite2.run(10)

Connect the single process to an emitter instance

In [15]:
from biosimulator_processes.data_model import EmitterConfig, EmittedType


emitter_config = EmitterConfig(
    emit=[
        EmittedType(value_name='floating_species', _type='tree[float]'),
        EmittedType(value_name='time', _type='float')
        
    ],
    name='emitter'
)

In [27]:
# b2[emitter_config.name].add_process(
#     name='ram-emitter',
#     emit={
#        'floating_species': 'tree[float]',
#        'time': 'float'}
# )
b2['emitter'].add_process(
    name='ram-emitter',
    emit={
       'floating_species': 'tree[float]',
       'time': 'float'}
)

In [28]:
b2.connect_all()

In [29]:
b2.visualize()

In [30]:
b2

In [31]:
composite3 = b2.generate()

In [32]:
composite3.run(10)

Save the composite to a file

In [33]:
b2.write(filename='single_copasi_process_with_emitter')

In [34]:
!ls out

In [35]:
!cat out/single_copasi_process_with_emitter.json

In [36]:
import json 


with open('out/single_copasi_process_with_emitter.json', 'r') as fp:
    data = json.load(fp)
    
    
data.keys()